# Caso de estudio: Análisis de sentimiento

En este ejercicio se usará la librería sklearn con el fin de crear un clasificador  que nos permita determinar el sentimiento del texto: positivo o negativo.

El dataset con el que vamos a trabajar podéis descargarlo desde la siguiente URL: 
https://www.kaggle.com/nehasontakke/amazon-unlocked-mobilecsv

## Carga del conjunto de datos
Para resolver este cuaderno de prácticas vamos a trabajar con el dataset *Amazon_Unlocked_Mobile.csv*, liberado por [PromptCloud](https://www.promptcloud.com/) y  que contiene información sobre opiniones, calificaciones y precios a cerca de 400.000 teléfonos móviles libres vendidos en Amazon.com

Para agilizar los cálculos podéis seleccionar una muestra de los datos. (*df = df.sample(frac=0.1, random_state=10)*)


In [1]:
import pandas as pd
import numpy as np

# Lectura de datos
file_path = './Amazon_Unlocked_Mobile.csv'

df = pd.read_csv(file_path) 

# Selección de la muestra
df = df.sample(frac=0.1, random_state=10)
df.dropna(inplace=True)

#A esto no le tenemos que hacer nada mas, es la carga de datos programada, solo poner el fichero dentro de la carpeta. 
df.head(5)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
34377,Apple iPhone 5c 8GB (Pink) - Verizon Wireless,Apple,194.99,1,"The phone needed a SIM card, would have been n...",1.0
248521,Motorola Droid RAZR MAXX XT912 M Verizon Smart...,Motorola,174.99,5,I was 3 months away from my upgrade and my Str...,3.0
167661,CNPGD [U.S. Office Extended Warranty] Smartwat...,CNPGD,49.99,1,an experience i want to forget,0.0
73287,Apple iPhone 7 Unlocked Phone 256 GB - US Vers...,Apple,922.00,5,GREAT PHONE WORK ACCORDING MY EXPECTATIONS.,1.0
277158,Nokia N8 Unlocked GSM Touch Screen Phone Featu...,Nokia,95.00,5,I fell in love with this phone because it did ...,0.0


## Pregunta 1 
### Etiqueta el dataset (1,25 puntos)

Debes crear la variable Sentimiento. Para ello, debemos eliminar los registros de sentimiento neutral que se corresponden a los registros de Rating = 3. Después, se considerará sentimiento positivo (sentiment = 1) a los registros con rating > 3. El resto de registros tendrán un sentimiento negativo (sentiment = 0). *La variable sentiment debe ser numérica*

Muestra por pantalla las dimensiones del dataset.

In [2]:
df2=df.drop(df[df.Rating == 3].index)

df2.loc[df2['Rating'] < 3, 'Sentimiento'] = 0 
df2.loc[df2['Rating'] > 3, 'Sentimiento'] = 1
#Localizamos los Ratings igual a 3 y los borramos con drop.
#Con loc, asignamos el valor a la columna Sentimiento segun sea mayor o menor a 3.
df2.shape #Tenemos 30737 datos con 7 columnas

(30737, 7)

In [3]:
df2.head(5) #Se nos crea bien la variable sentimiento. 

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Sentimiento
34377,Apple iPhone 5c 8GB (Pink) - Verizon Wireless,Apple,194.99,1,"The phone needed a SIM card, would have been n...",1.0,0.0
248521,Motorola Droid RAZR MAXX XT912 M Verizon Smart...,Motorola,174.99,5,I was 3 months away from my upgrade and my Str...,3.0,1.0
167661,CNPGD [U.S. Office Extended Warranty] Smartwat...,CNPGD,49.99,1,an experience i want to forget,0.0,0.0
73287,Apple iPhone 7 Unlocked Phone 256 GB - US Vers...,Apple,922.00,5,GREAT PHONE WORK ACCORDING MY EXPECTATIONS.,1.0,1.0
277158,Nokia N8 Unlocked GSM Touch Screen Phone Featu...,Nokia,95.00,5,I fell in love with this phone because it did ...,0.0,1.0


## Pregunta 2
### Creación de dataset de entrenamiento y test (1,25 puntos)

Separa el dataset de la pregunta anterior en dos partes (entrenamiento y test) con la proporción 70-30. Muestra el número de filas del dataset de train y el de test.

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df2[['Product Name','Brand Name','Price',
                                                         'Rating','Reviews','Review Votes']],df2['Sentimiento'],test_size=0.3)
#De momento, mantenemos todas las columnas. Ya filtraremos despues la unica que nos interesa, Reviews. 
print(x_train.shape) #Efectivamente, el train es 70% del total y el test 30% del total.
print(x_test.shape)

(21515, 6)
(9222, 6)


In [6]:
x_train.head(10)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
388248,Sony Ericsson Xperia Play R800i Unlocked Phone...,Sony Ericsson,83.88,5,aguanta la pela!! buen producto!! tienda respo...,2.0
389200,Sony Xperia E C1504 Unlocked Android Phone--U....,Sony,34.99,5,"This is my first time using android, and i'm l...",43.0
293188,"Polaroid A5PK 5"" Unlocked Smartphone, No Contr...",Polaroid,79.12,2,Such a disappointment. I had high hopes for th...,1.0
103153,BLU Advance 4.0L Unlocked Smartphone -Global G...,BLU,149.99,5,I recommend it 100%,0.0
11232,Apple iPhone 4S 16GB (Black) - Locked to Sprint,Apple,53.00,4,Has worked perfectly but did not totally seem ...,0.0
201182,Huawei Mate 2 - Factory Unlocked (Black),Huawei,229.99,5,For the price this is a great phone! It has al...,2.0
2691,ALCATEL OneTouch Idol 3 Global Unlocked 4G LTE...,Alcatel,129.00,5,I can never seem to settle on a phone. I splur...,6.0
128453,"BLU Life Pure Unlocked Phone 32GB ROM, 2GB RAM...",BLU,150.00,4,Good phone.,0.0
112681,"BLU Dash JR 4.0K Android 4.2, 2MP - Unlocked (...",BLU,49.00,4,It was really stressful at first to remove the...,0.0
362262,Samsung Galaxy S6 Edge G925i 32GB Unlocked GSM...,Samsung,519.99,5,I loved it.,0.0


In [7]:
x_test.head(10)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
293644,"Polaroid A6BK 6"" Unlocked Smartphone, No Contr...",Polaroid,107.52,2,It's huge!!! Way too big! Even bigger than the...,1.0
287678,OtterBox Rugged Defender Series for iPhone 5/5...,OtterBox,39.00,5,Exactly what I wanted and expected. Terrific p...,0.0
44420,Apple iPhone 5s 32GB (Silver) - AT&T,Apple,43.95,5,Great!!,0.0
167082,CNPGD [U.S. Office Extended Warranty] Smartwat...,CNPGD,49.99,4,Just received the watch so far so good connect...,0.0
389284,Sony Xperia E C1504 Unlocked Android Phone--U....,Sony,34.99,2,I used it for a week. had to return it. Phone ...,23.0
266996,Nokia Lumia 1020 32GB Unlocked GSM Phone w/ 41...,Nokia,296.00,2,I returned this product. Because flash was not...,0.0
78905,Apple Smart Watch Sport 38mm - Stainless Steel...,Apple,269.99,5,"Can't even tell it's refurbished, works like a...",4.0
392537,Sony Xperia Z C6602 Unlocked Phone with 5 inch...,Sony,169.99,5,excelente,0.0
381858,Samsung S5230 Hello Kitty Pink Unlocked GSM Qu...,Samsung,79.00,5,This is a really cool phone. My step-daughter ...,9.0
13137,Apple iPhone 4S 16GB Unlocked GSM - White (Cer...,Apple,129.99,1,Not factory unlock not working gsm carriers,1.0


In [8]:
y_train.head(10)

388248    1.0
389200    1.0
293188    0.0
103153    1.0
11232     1.0
201182    1.0
2691      1.0
128453    1.0
112681    1.0
362262    1.0
Name: Sentimiento, dtype: float64

In [9]:
y_test.head(10)

293644    0.0
287678    1.0
44420     1.0
167082    1.0
389284    0.0
266996    0.0
78905     1.0
392537    1.0
381858    1.0
13137     0.0
Name: Sentimiento, dtype: float64

## Pregunta 3
### Entrenamiento del dataset (1,25 puntos)

A continuación debemos utilizar el método de bolsas de palabras y transformar los documentos en una document term matrix.

*Utiliza para ello la función fit() y transform()*

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
#Solo analizamos donde haya emociones, los Review. Creamos la matriz de documentos para train y test.
x_train_matrix = vectorizer.fit_transform(x_train['Reviews']) #Hacemos fit y transform
x_test_matrix = vectorizer.transform(x_test['Reviews']) #SOLO hacemos transform

print(vectorizer.get_feature_names())

['00', '000', '0000', '000000', '000mah', '002', '00it', '00k', '00pm', '00us', '01', '013287002557427', '016', '02', '03', '04', '0412', '05', '06', '0630', '07', '07gb', '08', '09', '0_150511', '0an', '0ghz', '0hd', '0in', '0l', '0mp', '0social', '0v', '10', '100', '1000', '10000', '100000', '10000mah', '1001', '100gb', '100hours', '100mb', '100min', '101', '102', '1020', '1024', '103', '104', '1053', '106', '1060', '1067', '106miles', '107', '1080', '1080p', '109', '10gb', '10m', '10mbs', '10memory', '10music', '10navigation', '10other', '10ovi', '10phone', '10pm', '10th', '10touch', '10uscw52ena001558', '10w', '10x', '10year', '11', '110', '110v', '111', '112', '114', '115', '116', '118', '119', '11gb', '11pm', '11th', '11yeqr', '11yr', '12', '120', '1200', '12000mah', '120fps', '120h', '120v', '123', '124', '124gb', '125', '128', '1280x720', '128g', '128gb', '128gb32gb', '128m', '128sd', '129', '1296592924', '12feb', '12gb', '12hrs', '12mn', '12mp', '12mpx', '12th', '13', '130', '

In [11]:
x_train_matrix

<21515x18932 sparse matrix of type '<class 'numpy.int64'>'
	with 566445 stored elements in Compressed Sparse Row format>

In [12]:
x_test_matrix

<9222x18932 sparse matrix of type '<class 'numpy.int64'>'
	with 246580 stored elements in Compressed Sparse Row format>

## Pregunta 4
### Modelos Naive Bayes (1,25 puntos)

Declara y entrena el modelo de Naive Bayes con el conjunto de datos de entrenamiento. Para evaluar el clasificador debes predecir las etiquetas en el conjunto de datos test.
Como resultado muestra por pantalla los 10 primeros valores de la predicción sobre el conjunto de test.

In [13]:
from sklearn.naive_bayes import MultinomialNB
clfrNB = MultinomialNB()
clfrNB.fit(x_train_matrix, y_train)
predicted_labels = clfrNB.predict(x_test_matrix.toarray())
#Valores predichos
predicted_labels[0:10]

array([0., 1., 1., 1., 0., 0., 1., 1., 1., 0.])

In [14]:
#Valores reales
y_test[0:10]

293644    0.0
287678    1.0
44420     1.0
167082    1.0
389284    0.0
266996    0.0
78905     1.0
392537    1.0
381858    1.0
13137     0.0
Name: Sentimiento, dtype: float64

In [15]:
from sklearn.metrics import accuracy_score   #Obtenemos la prediccion concreta 
accuracy_score(y_test,predicted_labels) #Lo hacemos en el Bloque 4, pero ponerlo aqui esta bien para tener un orden de magnitud.

0.9211667751030145

## Pregunta 5
### Modelos Regresión logística (1,25 puntos)

Declara y entrena el modelo de Regresicón logística con el conjunto de datos de entrenamiento. Para evaluar el clasificador debes predecir las etiquetas en el conjunto de datos de test.
Como resultado muestra por pantalla los 10 primeros valores de la predicción sobre el conjunto de test.

In [16]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(x_train_matrix, y_train)
predicted_labels2 = classifier.predict(x_test_matrix.toarray())
predicted_labels2[0:10]

C:\Users\inima\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0., 1., 1., 1., 0., 0., 1., 1., 1., 0.])

In [17]:
#Valores reales
y_test[0:10]

293644    0.0
287678    1.0
44420     1.0
167082    1.0
389284    0.0
266996    0.0
78905     1.0
392537    1.0
381858    1.0
13137     0.0
Name: Sentimiento, dtype: float64

In [18]:
accuracy_score(y_test,predicted_labels2 )#Lo hacemos en el Bloque 4, pero ponerlo aqui esta bien para tener un orden de magnitud

0.9360225547603557

## Pregunta 6
### Modelos SVM (1,25 puntos)

Declara y entrena el modelo SVM con el conjunto de datos de entrenamiento. Para evaluar el clasificador debes predecir las etiquetas en el conjunto de datos de test.
Como resultado muestra por pantalla los 10 primeros valores de la predicción sobre el conjunto de test.


In [19]:
from sklearn import svm
clfrSVM = svm.SVC(kernel='linear', C=0.1)
clfrSVM.fit(x_train_matrix, y_train)
predicted_labels3 = clfrSVM.predict(x_test_matrix.toarray())
predicted_labels3[0:10]

array([0., 1., 1., 1., 0., 0., 1., 1., 1., 0.])

In [20]:
#Valores reales
y_test[0:10]

293644    0.0
287678    1.0
44420     1.0
167082    1.0
389284    0.0
266996    0.0
78905     1.0
392537    1.0
381858    1.0
13137     0.0
Name: Sentimiento, dtype: float64

In [21]:
accuracy_score(y_test,predicted_labels3)#Lo hacemos en el Bloque 4, pero ponerlo aqui esta bien para tener un orden de magnitud.

0.9322272825851226

## Pregunta 7
### Ajuste de los parámetros de la entidad countvectorizer (1,25 puntos)

Ajusta la entidad *countvectorizer* a los datos de entrenamiento especificando una frecuencia mínima de documentos de 5 y extrayendo unigramas y bigramas (min_df, ngram_range) y entrena de nuevo el modelo de Naive Bayes.
Como resultado muestra por pantalla los 10 primeros valores de la predicción sobre el conjunto de test.

In [22]:
vectorizer2 = CountVectorizer(min_df = 5,ngram_range=(1, 2))
#Solo analizamos donde haya emociones, los Review
x_train_matrix2 = vectorizer2.fit_transform(x_train['Reviews']) #Hacemos fit y transform
x_test_matrix2 = vectorizer2.transform(x_test['Reviews']) #SOLO hacemos transform

print(vectorizer2.get_feature_names())

['00', '00 and', '00 for', '00 price', '00 the', '00 to', '000', '01', '02', '04', '09', '10', '10 10', '10 15', '10 and', '10 days', '10 for', '10 hours', '10 mins', '10 minutes', '10 months', '10 of', '10 on', '10 phone', '10 seconds', '10 the', '10 times', '10 to', '10 would', '10 year', '10 years', '100', '100 00', '100 and', '100 for', '100 in', '100 ok', '100 phone', '100 recomendado', '100 recommended', '100 satisfaction', '100 satisfied', '100 to', '100 when', '1000', '1020', '1080p', '10th', '11', '110', '11th', '12', '12 hours', '12 year', '120', '125', '128', '128 gb', '128gb', '129', '13', '13 mp', '13 year', '130', '13mp', '13mp camera', '14', '14 days', '14 hours', '140', '1440p', '15', '15 20', '15 and', '15 days', '15 minutes', '15 seconds', '15 years', '150', '1520', '16', '16 and', '16 gb', '16 gig', '16 mp', '160', '16g', '16gb', '16gb microsd', '16gb of', '16gb version', '16mp', '17', '1700', '18', '180', '1800', '19', '190', '1900', '199', '1gb', '1gb of', '1gb ram

In [23]:
clfrNB2 = MultinomialNB()
clfrNB2.fit(x_train_matrix2, y_train)
predicted_labels4 = clfrNB2.predict(x_test_matrix2.toarray())
#Valores predichos
predicted_labels4[0:10]

array([0., 1., 1., 1., 0., 0., 1., 1., 1., 0.])

In [24]:
#Valores reales
y_test[0:10]

293644    0.0
287678    1.0
44420     1.0
167082    1.0
389284    0.0
266996    0.0
78905     1.0
392537    1.0
381858    1.0
13137     0.0
Name: Sentimiento, dtype: float64

In [25]:
accuracy_score(y_test,predicted_labels4) #La precision ha aumentado ligeramente.

0.9342875731945348

## Pregunta 8
### Ajuste de los parámetros de la entidad tfidfvectorizer (1,25 puntos)

Utiliza en vez de la entidad *countvectorizer* la entidad *tfidfvectorizer* especificando una frecuencia mínima de documentos de 5 y extrayendo también unigramas y bigramas y entrena de nuevo el modelo de Regresión logística. Como resultado muestra por pantalla los 10 primeros valores de la predicción sobre el conjunto de test.

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer3 = TfidfVectorizer(min_df = 5,ngram_range=(1, 2))
#Solo analizamos donde haya emociones, los Review
x_train_matrix3 = vectorizer3.fit_transform(x_train['Reviews']) #Hacemos fit y transform
x_test_matrix3 = vectorizer3.transform(x_test['Reviews']) #SOLO hacemos transform

print(vectorizer3.get_feature_names())

['00', '00 and', '00 for', '00 price', '00 the', '00 to', '000', '01', '02', '04', '09', '10', '10 10', '10 15', '10 and', '10 days', '10 for', '10 hours', '10 mins', '10 minutes', '10 months', '10 of', '10 on', '10 phone', '10 seconds', '10 the', '10 times', '10 to', '10 would', '10 year', '10 years', '100', '100 00', '100 and', '100 for', '100 in', '100 ok', '100 phone', '100 recomendado', '100 recommended', '100 satisfaction', '100 satisfied', '100 to', '100 when', '1000', '1020', '1080p', '10th', '11', '110', '11th', '12', '12 hours', '12 year', '120', '125', '128', '128 gb', '128gb', '129', '13', '13 mp', '13 year', '130', '13mp', '13mp camera', '14', '14 days', '14 hours', '140', '1440p', '15', '15 20', '15 and', '15 days', '15 minutes', '15 seconds', '15 years', '150', '1520', '16', '16 and', '16 gb', '16 gig', '16 mp', '160', '16g', '16gb', '16gb microsd', '16gb of', '16gb version', '16mp', '17', '1700', '18', '180', '1800', '19', '190', '1900', '199', '1gb', '1gb of', '1gb ram

In [27]:
classifier = LogisticRegression()
classifier.fit(x_train_matrix3, y_train)
predicted_labels5 = classifier.predict(x_test_matrix3.toarray())
predicted_labels5[0:10]

array([0., 1., 1., 1., 0., 0., 1., 1., 1., 0.])

In [28]:
#Valores reales
y_test[0:10]

293644    0.0
287678    1.0
44420     1.0
167082    1.0
389284    0.0
266996    0.0
78905     1.0
392537    1.0
381858    1.0
13137     0.0
Name: Sentimiento, dtype: float64

In [29]:
accuracy_score(y_test,predicted_labels5) #La precision aumenta muy ligeramente

0.9378659726740404

Ese seria el fin del ejercicio 3.